In [ ]:
import pandas as pd
#import numerapi
import sklearn.linear_model

# import keras_deepCNN
# from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
import time

# gradient boosting for classification in scikit-learn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.externals import joblib
import xgboost as xgb

In [ ]:
useValidationData=True

tic = time.time()
# training_data = pd.read_csv("balancedTrainingData_test12021-01-31_2021-03-21.csv")

# feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
# X_train = training_data[feature_cols].to_numpy()

# #Y_train = training_data.loc[:,['label_0', 'label_025', 'label_05', 'label_075', 'label_1']].to_numpy()
# Y_train = training_data.loc[:,['target']].to_numpy()

#training_data = pd.read_csv("appendedTrainingData_test12021-02-28_2021-03-28.csv")
training_data = pd.read_csv("data/numerai_datasets_28.03.21/numerai_training_data.csv")

feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
X_train = training_data[feature_cols].reset_index().drop(['index'], axis = 1)


X_train_0 = X_train.to_numpy()
Y_train_0 = training_data.target
# Y_train_0 = Y_train_0.replace(0.5, 2)
# Y_train_0 = Y_train_0.replace([0.25, 0, 0.75, 1], 0)
# Y_train_0 = Y_train_0.replace(2, 1)
Y_train_0 = Y_train_0.to_numpy()

if useValidationData:
    tournament_data = pd.read_csv("data/numerai_datasets_28.03.21/numerai_tournament_data.csv")
    validation_data = tournament_data.loc[tournament_data.data_type == 'validation']
    X_val = validation_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
    Y_val = validation_data.target.to_numpy()
    Y_val_0 = Y_val
    X_val_0 = X_val
    # Y_val_0 = Y_val.replace(0.5, 2)
    # Y_val_0 = Y_val_0.replace([0.25, 0, 0.75, 1], 0)
    # Y_val_0 = Y_val_0.replace(2, 1)

toc = time.time()
print("loaded the data took ", toc - tic)

In [ ]:
Y_train_0 = training_data.target
# 0, 1       -> 0
# 0.5        -> 1
# 0.25, 0.75 -> 2

# Y_train_0 = Y_train_0.replace([0.25, 0.75], 2)
# Y_train_0 = Y_train_0.replace([0, 1], 0)
# Y_train_0 = Y_train_0.replace(0.5, 1)
#Y_train_0 = Y_train_0.replace(2, 1)

feature_cols_wisdom = training_data.columns[training_data.columns.str.startswith('feature_w')]
feature_cols_consti = training_data.columns[training_data.columns.str.startswith('feature_con')]
feature_cols_streng = training_data.columns[training_data.columns.str.startswith('feature_str')]
feature_cols_charis = training_data.columns[training_data.columns.str.startswith('feature_cha')]
feature_cols_intell = training_data.columns[training_data.columns.str.startswith('feature_int')]

X_train = training_data[feature_cols].reset_index().drop(['index'], axis = 1)

Y_val_0 = validation_data.target#.replace([0.25, 0.75], 2)
# Y_val_0 = Y_val_0.replace([0, 1], 0)
# Y_val_0 = Y_val_0.replace(0.5, 1)

X_val = validation_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()


In [ ]:
#X_train, Y_train = shuffle(X_train.to_numpy(), training_data.target.to_numpy())

X_train, Y_train = shuffle(X_train.to_numpy(), Y_train_0.to_numpy())
Y_val = Y_val_0.to_numpy()

# le = LabelEncoder()
# Y_train = le.fit_transform(Y_train)
# Y_val = le.fit_transform(Y_val)

In [ ]:
from sklearn.utils import class_weight
#classes = np.array([0, 1, 2])
classes = np.array([0, 0.25, 0.5, 0.75, 1])
class_weights = class_weight.compute_class_weight('balanced', classes, Y_train)
#class_weights = dict(enumerate(class_weights))

w_array = np.ones(Y_train.shape[0], dtype = 'float')
for i, val in enumerate(Y_train):
    index = np.where(classes == val)
    w_array[i] = class_weights[index]
    
evalset = [(X_train, Y_train), (X_val,Y_val)]

In [ ]:
model = XGBClassifier(
        #base_score=0.0, booster='gbtree', colsample_bylevel=1,
    #    colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
    #    max_delta_step=0.0, max_depth=8, min_child_weight=1, missing=None,
        n_estimators=1000, #n_jobs=10, nthread=None,
    #    objective='multi:softprob',# random_state=0, 
    #    reg_alpha= 0, reg_lambda = 0,
    #    reg_lambda=1.0, scale_pos_weight=1, seed=None, silent=None,
    #    subsample=0.8, verbosity=0
     #   eta = 0.05,
        #subsample=0.5, colsample_bytree=0.5, colsample_bylevel=0.5,
#         max_depth = 20,
#         gamma = 10,
#         min_child_weight = 5
    colsample_bytree=0.1
       ) 
model.fit(X_train, Y_train,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=50) #xgb_model='model_kFold.model'

In [ ]:
3 classes + default xgb
[1471]	validation_0-mlogloss:0.52806	validation_1-mlogloss:0.96262
3 classes + only wisdom feature
[762]	validation_0-mlogloss:0.86781	validation_1-mlogloss:1.02326
        
5 classes + only wisdom
[1615]	validation_0-mlogloss:1.05126	validation_1-mlogloss:1.49122

In [ ]:
model.save_model('model_938Train_406Val.model')  #val loss end 1.46256, mit gamma = 0, loss = 1.465, gamma = 10, loss = 1.447

# gamma = 0, depth = 10
# [100]	validation_0-mlogloss:0.65688	validation_1-mlogloss:1.36347
# gamma = 20, min_child_weight = 1
# [100]	validation_0-mlogloss:1.41976	validation_1-mlogloss:1.47285
# gamma = 20, min_child_weight = 2
# [100]	validation_0-mlogloss:1.42026	validation_1-mlogloss:1.47273  diff = 0.0521
# gamma = 20, min_child_weight = 5      
# [100]	validation_0-mlogloss:1.41991	validation_1-mlogloss:1.47209  diff = 0.0529
# gamma = 20, min_child_weight = 1.5
# [100]	validation_0-mlogloss:1.41930	validation_1-mlogloss:1.47213

# #gamma = 2
# [100]	validation_0-mlogloss:1.40372	validation_1-mlogloss:1.48010
# #gamma = 1
# [100]	validation_0-mlogloss:1.40386	validation_1-mlogloss:1.47948
# #gamma = 0.5
# [100]	validation_0-mlogloss:1.40408	validation_1-mlogloss:1.48020
# #gamma = 0.75
# [100]	validation_0-mlogloss:1.40383	validation_1-mlogloss:1.47926
# #gamma = 0.825
# [100]	validation_0-mlogloss:1.40383	validation_1-mlogloss:1.47926
        
# #min_child_weight = 2
# [100]	validation_0-mlogloss:1.40407	validation_1-mlogloss:1.48185
# #min_child_weight = 0.75
# [100]	validation_0-mlogloss:1.40398	validation_1-mlogloss:1.48077
# #min_child_weight = 0.75       
# [100]	validation_0-mlogloss:1.40387	validation_1-mlogloss:1.48020
# #min_child_weight = 1.25       
# [100]	validation_0-mlogloss:1.40376	validation_1-mlogloss:1.48066
# #min_child_weight = 5 
# [100]	validation_0-mlogloss:1.40439	validation_1-mlogloss:1.48035
# #min_child_weight = 50     
# [100]	validation_0-mlogloss:1.40725	validation_1-mlogloss:1.48065
# #min_child_weight = 10
# [100]	validation_0-mlogloss:1.40439	validation_1-mlogloss:1.48035
# #min_child_weight = 0.5      
# [100]	validation_0-mlogloss:1.40379	validation_1-mlogloss:1.48014
# #0.2
# [100]	validation_0-mlogloss:1.40377	validation_1-mlogloss:1.48040
# #1.1
# [100]	validation_0-mlogloss:1.40363	validation_1-mlogloss:1.48024
# 1
# [100]	validation_0-mlogloss:1.40383	validation_1-mlogloss:1.47926
# 0.9
# [100]	validation_0-mlogloss:1.40350	validation_1-mlogloss:1.48004
# [400]	validation_0-mlogloss:1.16978	validation_1-mlogloss:1.44122
        
# lambda = 0.1, alpha = 0.1
# [100]	validation_0-mlogloss:1.40338	validation_1-mlogloss:1.48019
# lambda = 1, alpha = 1
# [100]	validation_0-mlogloss:1.40506	validation_1-mlogloss:1.47972
# lambda = 10, alpha = 10
# [100]	validation_0-mlogloss:1.41773	validation_1-mlogloss:1.48155
# lambda = 10, alpha = 2
# [100]	validation_0-mlogloss:1.41656	validation_1-mlogloss:1.48019
# lambda = 1, alpha = 0.5
# [100]	validation_0-mlogloss:1.40506	validation_1-mlogloss:1.47954
# lambda = 0, alpha = 1
# [100]	validation_0-mlogloss:1.40475	validation_1-mlogloss:1.47962
# lambda = 0, alpha = 0
# [100]	validation_0-mlogloss:1.40357	validation_1-mlogloss:1.47998

In [ ]:
results = model.evals_result()
# plot learning curves
pyplot.plot(results['validation_0']['mlogloss'], label='train')
pyplot.plot(results['validation_1']['mlogloss'], label='test')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
Y_train_pred = model.predict(X_train)
corVal = 0
allTrainSamples = len(Y_train_pred)
for i in range(allTrainSamples):
    if Y_train_pred[i] == Y_train[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("train accuracy = ", accuracy)


y_pred = model.predict(X_val)
corVal = 0
allTrainSamples = len(y_pred)
for i in range(allTrainSamples):
    if y_pred[i] == Y_val[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("validation accuracy = ", accuracy)

In [ ]:
#100 classifiers + no weights
Y_train_pred = model.predict(X_train)
corVal = 0
allTrainSamples = len(Y_train_pred)
for i in range(allTrainSamples):
    if Y_train_pred[i] == Y_train[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("train accuracy = ", accuracy)


y_pred = model.predict(X_val)
corVal = 0
allTrainSamples = len(y_pred)
for i in range(allTrainSamples):
    if y_pred[i] == Y_val[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("validation accuracy = ", accuracy)

In [ ]:
#1000 classifiers + weights
Y_train_pred = model.predict(X_train)
corVal = 0
allTrainSamples = len(Y_train_pred)
for i in range(allTrainSamples):
    if Y_train_pred[i] == Y_train[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("train accuracy = ", accuracy)


y_pred = model.predict(X_val)
corVal = 0
allTrainSamples = len(y_pred)
for i in range(allTrainSamples):
    if y_pred[i] == Y_val[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("validation accuracy = ", accuracy)

In [ ]:
#1000 classifiers + no weights
Y_train_pred = model.predict(X_train)
corVal = 0
allTrainSamples = len(Y_train_pred)
for i in range(allTrainSamples):
    if Y_train_pred[i] == Y_train[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("train accuracy = ", accuracy)


y_pred = model.predict(X_val)
corVal = 0
allTrainSamples = len(y_pred)
for i in range(allTrainSamples):
    if y_pred[i] == Y_val[i]:
        corVal +=1
accuracy = corVal / allTrainSamples

print("validation accuracy = ", accuracy)

In [ ]:
tournament_data_recent = pd.read_csv("data/numerai_datasets_28.03.21/numerai_tournament_data.csv")
validation_data_recent = tournament_data_recent.loc[tournament_data_recent.data_type == 'validation']
X_val_r = validation_data_recent[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
Y_val_r = validation_data_recent.target
Y_val_r = Y_val_r.to_numpy()

In [ ]:
#5000 classifiers + no weights
# Y_train_pred = model.predict(X_train)
# corVal = 0
# allTrainSamples = len(Y_train_pred)
# for i in range(allTrainSamples):
#     if Y_train_pred[i] == Y_train[i]:
#         corVal +=1
# accuracy = corVal / allTrainSamples

# print("train accuracy = ", accuracy)


y_pred = model.predict(X_val_r)
corVal = 0
allTrainSamples = len(y_pred)
for i in range(allTrainSamples):
    if y_pred[i] == Y_val_r[i]:
        corVal +=1
#     else:
#         print("predicted = ", y_pred[i], "ground truth = ", Y_val[i])
accuracy = corVal / allTrainSamples

print("validation accuracy = ", accuracy)

In [ ]:
predictions_df = validation_data["id"].to_frame()
predictions_df["pred"] = y_pred
predictions_df.pred.hist(bins = 10)

In [ ]:
training_df = training_data["id"].to_frame()
training_df["train_pred"] = Y_train_pred
training_df.train_pred.hist(bins = 10)

In [ ]:
validation_data.target.hist(bins=10)

In [ ]:
#validation score
y_pred = model.predict(X_val)

scores = accuracy_score(Y_val_0, y_pred)
print("crossvalidation score", scores)

cm = metrics.confusion_matrix(Y_val_0, y_pred)
print("confusion matrix", cm)
cr = metrics.classification_report(Y_val_0, y_pred)
print("classification report", cr)

In [ ]:
# Save to file in the current working directory
joblib_file = "joblib_model_gradientBoosting_99train_46dev.pkl"
joblib.dump(model, joblib_file)

In [ ]:
X_tain